In [1]:
import os
import numpy
from pathlib import Path

# ==========================================
# 🚀 3D SCANNER: THE FINAL BOSS FIX (Numpy 2.0)
# ==========================================

print("🧹 1. เคลียร์พื้นที่และโหลดโค้ดใหม่...")
if os.path.exists("3DSCAN"):
    !rm -rf 3DSCAN
!git clone https://github.com/PRIDA-TAKON/3DSCAN.git
os.chdir("3DSCAN")

# ---------------------------------------------------------

# =========================================================
# 🚑 EMERGENCY: NUCLEAR RESET & SANITIZER
# =========================================================

print("\n🧹 Sanitizing 3d-scan.py (Preventing re-infection)...")
import os
try:
    target_file = "3d-scan.py"
    if os.path.exists(target_file):
        with open(target_file, "r") as f:
            code = f.read()
        
        modified = False
        if "patch_numpy()" in code:
            # Rename definition to prevent matching
            code = code.replace("def patch_numpy():", "def patch_numpy_DISABLED():")
            # Disable the call
            code = code.replace("patch_numpy()", "# patch_numpy() disabled by notebook")
            print("   ✅ Harmful patch removed from 3d-scan.py")
            modified = True
            
        if "numpy<2.0" in code:
            code = code.replace('"numpy<2.0"', '"numpy"')
            print("   ✅ Numpy < 2.0 restriction removed from 3d-scan.py")
            modified = True
            
        if modified:
            with open(target_file, "w") as f:
                f.write(code)
        else:
            print("   ✅ No changes needed.")
    else:
        print(f"   ⚠️ Error: {target_file} not found locally.")
except Exception as e:
    print(f"   ⚠️ Warning Sanitizing: {e}")

# FIX 2: แก้ปัญหา np.round_ (บอสตัวใหม่ Numpy 2.0)
# ---------------------------------------------------------
print("\n🛡️ 3. ฝังยาแก้ปวดที่ 2 (Compatibility Patch for np.round_)...")
# เราจะฉีดโค้ดนี้เข้าไปที่บรรทัดแรกสุดของไฟล์ 3d-scan.py เลย
# เพื่อให้มั่นใจว่า Numpy ถูกแก้ก่อนที่ library อื่นจะถูกโหลด
compatibility_code = """
import numpy
import sys
# --- KAGGLE HOTFIX 2 (Numpy 2.0) ---
try:
    if not hasattr(numpy, "round_"):
        numpy.round_ = numpy.round
        print("🩹 Patched numpy.round_ -> numpy.round (Legacy Support Enabled)")
except Exception as e:
    print(f"⚠️ Patch failed: {e}")
# -----------------------------------
"""

# หาไฟล์ python หลัก
import glob
py_files = glob.glob("*.py")
target_script = py_files[0] if py_files else "3d-scan.py"

# อ่านไฟล์เดิม
with open(target_script, "r") as f:
    original_script = f.read()

# เขียนทับ: เอาโค้ดแก้ไปแปะไว้บนสุด + ตามด้วยโค้ดเดิม
with open(target_script, "w") as f:
    f.write(compatibility_code + "\n" + original_script)
print("✅ Patch np.round_ สำเร็จ (แปะไว้บนสุดของไฟล์)")

# ---------------------------------------------------------
# FIX 3: ปิด GPU Matching (กัน Crash)
# ---------------------------------------------------------
print("\n🔧 4. ปิด GPU Matching...")
!sed -i 's/"--SiftMatching.use_gpu", "1"/"--SiftMatching.use_gpu", "0"/' *.py
!sed -i 's/"--SiftExtraction.use_gpu", "1"/"--SiftExtraction.use_gpu", "0"/' *.py

# ---------------------------------------------------------
print(f"\n🚀 5. เริ่มรัน {target_script} ...")
print("-" * 50)
!python "{target_script}" 2>&1 | tee 3d-scan.log.txt